In [83]:
# inner module import
import sys
sys.path.append("/storage/homefs/yc24j783/datacat4ml/datacat4ml")
from const import FETCH_DATA_DIR, FETCH_FIG_DIR, FEATURIZE_DATA_DIR, FEATURIZE_FIG_DIR
# print current work directory
workdir = os.getcwd()


import os
import pandas as pd

In [65]:
# load featurized data
ki_mor_1_df = pd.read_pickle(os.path.join(FEATURIZE_DATA_DIR, 'ki_maxcur', 'ki_target_CHEMBL233_1_fp.pkl'))

In [66]:
print(f'The shape of ki_mor_1_df is {ki_mor_1_df.shape}')
ki_mor_1_df.head(1)

The shape of ki_mor_1_df is (298, 31)


,assay_id,assay_chembl_id,tid,target_chembl_id,standard_type,pchembl_value,assay_type,assay_category,assay_organism,assay_tax_id,...,relationship_type,aidx,confidence_score,molregno,compound_chembl_id,canonical_smiles,assay_info_hash,ecfp4,map4c,activity
0,148100,CHEMBL751582,129,CHEMBL233,Ki,4.97,B,None,None,None,...,H,CLD0,8,176210,CHEMBL108417,CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1,901e030c965234752af5ce2b09ef2dea,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9923915, 4152229, 2453811, 1604050, 9383906, ...",inactive


In [67]:
ki_mor_1_df['assay_id'].value_counts()

assay_id
148100    33
148222    22
148248    21
148089    20
495338    19
458378    18
148088    17
148097    17
878883    15
149492    12
563750    11
767030    10
611744     9
148229     7
148226     7
559849     6
753181     6
753180     6
753179     6
148102     6
138858     5
448535     4
148841     4
753173     3
148093     2
148234     2
221913     1
149326     1
148236     1
148246     1
148095     1
148244     1
148245     1
148243     1
148094     1
148247     1
Name: count, dtype: int64

In [68]:
ki_mor_1_df['assay_desc'].value_counts()

assay_desc
Displacement of [3H]DAMGO from mu opioid receptor expressed in CHO cells                                                                                    44
Binding affinity at human opioid receptor mu 1 was determined by using [3H]diprenorphine radioligand in CHO cell membranes at a concentration of 0.12 nM    33
In vitro binding affinity to human Opioid receptor mu 1 on CHO cell membranes using [3H]diprenorphine displacement.                                         22
Inhibitory activity against Opioid receptor mu 1 in chinese Hamster Ovary (CHO) cells membranes was determined using [3H]-DAMGO radioligand                 21
Ability to displace [3H]DAMGO from human recombinant Opioid receptor mu 1 in CHO cells                                                                      20
Displacement of [3H]naloxone from monocloned mu opioid receptor expressed in CHO cells                                                                      19
Displacement of [3H]DAMGO from huma

# Step 1: Tokenization

## tokenize smiles

In [69]:
def regex_tokenizer(smi:str):
    """
    Tokenize a SMILES string using regular expression.

    The pros of this method is that it is simple and fast.

    :param smi: SMILES string

    :return: list of tokens
    """
    import re
    pattern =  "(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smi)]
    assert smi == ''.join(tokens)

    return ' '.join(tokens)

In [85]:
# set a new column 'regex_token_smi' to store the tokenized SMILES
ki_mor_1_df['regex_token_smi'] = ki_mor_1_df['canonical_smiles'].apply(regex_tokenizer)

In [71]:
def chemberta_tokenizer(smi: str, max_smi_len: int = 200, padding: bool = True, truncation: bool = True,
               auto_tokenizer: str = 'seyonec/PubChem10M_SMILES_BPE_450k'):
        """ Tokenize SMILES for a ChemBerta Transformer

        The pros of this method compared to the the one based on regular expression is that
        it leverages pre-trained knowledge for handling complex SMILES syntax like aromatic rings, nested branches, etc.

        :param max_smiles_length: (int) Maximal allowable SMILES string length
        :param padding: (bool) allow padding
        :param truncation: (bool) allow truncation (you will need this for heterogeneous SMILES strings)
        :param auto_tokenizer: (str) name of the auto tokenizer provided by HuggingFace

        :return: Dict['input_ids': tensor, 'attention_mask': tensor], tensors are of shape N x max_smiles_length
        """

        from transformers import AutoTokenizer

        chemical_tokenizer = AutoTokenizer.from_pretrained(auto_tokenizer)
        tokens = chemical_tokenizer(smi, padding=padding, truncation=truncation,max_length=max_smi_len,
                                   return_tensors='pt') # `return_tensors='pt'` means to convert the output to PyTorch tensors. Removing this argument will return a dictionary of lists.

        return tokens

In [72]:
#chemberta_token_smis = [chemberta_tokenizer(smi) for smi in smis]
#chemberta_token_smis[:3]

/storage/homefs/yc24j783/miniconda3/envs/enztrans_test/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'input_ids': tensor([[  0, 262,  12,  71,  21, 268,  21,  13,  50,  21, 262,  63,  39,  36,
           44,  65,  21,  63,  39,  36,  36,  44, 336,  50,  13,  71,  21, 276,
           12, 275,  13,  71,  21,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  0, 372,  12,  71,  21, 268,  21,  13,  50,  21, 262,  63,  39,  36,
           44,  65,  21,  63,  39,  36,  36,  44, 336,  50,  13,  71,  21, 276,
           12, 275,  13,  71,  21,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  0, 265,  21, 264,  12,  39,  12,  71,  22, 264,  12,  39,  13, 261,
           22,  13,  50,  22, 262,  63,  39,  36,  44,  65,  22,  63,  39,  36,
           44, 336,  50,  13,  71,  22, 276,  12, 275,  13,  71,  22,  13, 261,
           21,   2]]), 'at

## tokenize assay_info

### assay_id

### assay_desc

In [73]:
def sentence_tokenizer(sent:str):
    """
    Tokenize the sentence for assay description using Tokenizer from HuggingFace. Optimze for GPCR assay description.

    :param sent: (str)sentence to be tokenized
    """

    from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors

    # Initialize a tokenizer
    tokenizer = Tokenizer(models.BPE())

    # Customize pre-tokenization and decoding
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
    tokenizer.decoder = decoders.ByteLevel()
    tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)

    # And then train
    trainer = trainers.BpeTrainer(special_tokens=['affinity', 'displacement', '3H', '125I', 'camp', 'gtp', 'calcium', 'ca2+', 'IP1', 'IP3', 'arrest', 'agonist'])
    tokenizer.train_from_iterator([sent], trainer=trainer)

    encoded = tokenizer.encode(sent)
    my_list = [item for item in encoded.tokens if 'Ġ' != item]
    my_list = [item.replace('Ġ', '_') for item in my_list]
    my_list = ' '.join(my_list)

    return my_list

In [74]:
assay_descs = ki_mor_1_df['assay_desc'].values
sent_token_descs = [sentence_tokenizer(sent) for sent in assay_descs]
sent_token_descs[:3]

['_Binding affinity _at _human _opioid _receptor _mu _1 _was _determined _by _using _[ 3H ] diprenorphine _radioligand _in _CHO _cell _membranes _at _a _concentration _of _0 . 12 _nM',
 '_Binding affinity _at _human _opioid _receptor _mu _1 _was _determined _by _using _[ 3H ] diprenorphine _radioligand _in _CHO _cell _membranes _at _a _concentration _of _0 . 12 _nM',
 '_Binding affinity _at _human _opioid _receptor _mu _1 _was _determined _by _using _[ 3H ] diprenorphine _radioligand _in _CHO _cell _membranes _at _a _concentration _of _0 . 12 _nM']

### assay-related fields

## concatenate the tokenized SMILES and assay_info

## train test split

In [75]:
import random

# split the data using random sampling
ki_mor_1_df['set'] = ''
smis = list(ki_mor_1_df['canonical_smiles'].values)
random_20per = random.sample(smis, int(0.2*len(smis)))
random_10per_1 = random_20per[0:int(0.1*len(smis))]
random_10per_2 = random_20per[int(0.1*len(smis)):len(random_20per)]

# Assign the set
ki_mor_1_df.loc[ki_mor_1_df['canonical_smiles'].isin(random_10per_1), 'set'] = 'VAL'
ki_mor_1_df.loc[ki_mor_1_df['canonical_smiles'].isin(random_10per_2), 'set'] = 'TEST'
ki_mor_1_df.loc[ki_mor_1_df['set'] == '', 'set'] = 'TRAIN'

# check the distribution of the set
print(f'The length of the ki_mor_1_df is {len(ki_mor_1_df)}')
ki_mor_1_df['set'].value_counts()

The length of the ki_mor_1_df is 298


set
TRAIN    229
TEST      37
VAL       32
Name: count, dtype: int64

In [88]:
# define `train_src` is the list of value in the column 'canonical_smiles' where the column 'set' is 'TRAIN'
train_src = list(ki_mor_1_df.loc[ki_mor_1_df['set'] == 'TRAIN', 'regex_token_smi'].values)
# save the `train_src` to a text file
with open(os.path.join(workdir, 'enztrans_test', 'data', 'ki_mor_1', 'train_src.txt'), 'w') as f:
    for item in train_src:
        f.write("%s\n" % item)
val_src = list(ki_mor_1_df.loc[ki_mor_1_df['set'] == 'VAL', 'regex_token_smi'].values)
with open(os.path.join(workdir, 'enztrans_test', 'data', 'ki_mor_1', 'valid_src.txt'), 'w') as f:
    for item in val_src:
        f.write("%s\n" % item)
test_src = list(ki_mor_1_df.loc[ki_mor_1_df['set'] == 'TEST', 'regex_token_smi'].values)
with open(os.path.join(workdir, 'enztrans_test', 'data', 'ki_mor_1', 'test_src.txt'), 'w') as f:
    for item in test_src:
        f.write("%s\n" % item)

train_tgt = list(ki_mor_1_df.loc[ki_mor_1_df['set'] == 'TRAIN', 'pchembl_value'].values)
with open(os.path.join(workdir, 'enztrans_test', 'data', 'ki_mor_1', 'train_tgt.txt'), 'w') as f:
    for item in train_tgt:
        f.write("%s\n" % item)
val_tgt = list(ki_mor_1_df.loc[ki_mor_1_df['set'] == 'VAL', 'pchembl_value'].values)
with open(os.path.join(workdir, 'enztrans_test', 'data', 'ki_mor_1', 'valid_tgt.txt'), 'w') as f:
    for item in val_tgt:
        f.write("%s\n" % item)
test_tgt = list(ki_mor_1_df.loc[ki_mor_1_df['set'] == 'TEST', 'pchembl_value'].values)
with open(os.path.join(workdir, 'enztrans_test', 'data', 'ki_mor_1', 'test_tgt.txt'), 'w') as f:
    for item in test_tgt:
        f.write("%s\n" % item)

# Step 2: Preprocess the data

In [77]:
import preprocess
# print the path for module `preprocess`
print(preprocess.__file__)

/storage/homefs/yc24j783/OpenNMT-py/preprocess.py


In [81]:
import argparse
parser = argparse.ArgumentParser('Preprocess the data')

parser.add_argument('--train_src', required=True, help='Path(s) to the training source data')
parser.add_argument('--train_tgt', required=True, help='Path(s) to the training target data')
parser.add_argument('--val_src', help='Path(s) to the validation source data')
parser.add_argument('--val_tgt', help='Path(s) to the validation target data')

parser.add_argument('--save_data', required=True, help='Path to save the preprocessed data')
parser.add_argument('--src_seq_length', type=int, default=50, help='Maximal allowable source sequence length')
parser.add_argument('--tgt_seq_length', type=int, default=50, help='Maximal allowable target sequence length')

parser.add_argument('--src_vocab_size', type=int, default=50000, help='Size of the source vocabulary')
parser.add_argument('--tgt_vocab_size', type=int, default=50000, help='Size of the target vocabulary')
parser.add_argument('--share_vocab', action='store_true', help='share source and target vocabulary')
parser.add_argument('--lower', action='store_true', help='lowercase data')

# make a directory 'preprocess' in the current working directory
os.makedirs('preprocessed', exist_ok=True)

#args = parser.parse_args()

usage: Preprocess the data [-h] --train_src TRAIN_SRC --train_tgt TRAIN_TGT
                           [--val_src VAL_SRC] [--val_tgt VAL_TGT] --save_data
                           SAVE_DATA [--src_seq_length SRC_SEQ_LENGTH]
                           [--tgt_seq_length TGT_SEQ_LENGTH]
                           [--src_vocab_size SRC_VOCAB_SIZE]
                           [--tgt_vocab_size TGT_VOCAB_SIZE] [--share_vocab]
                           [--lower]
Preprocess the data: error: the following arguments are required: --train_src, --train_tgt, --save_data


SystemExit: 2

In [ ]:

preprocess.main(
    TRAIN_SRC=train_src,
    train_tgt=train_tgt,
    valid_src=val_src,
    valid_tgt=val_tgt,
    save_data='preprocess',
    src_seq_length=3000, 
    tgt_seq_length=3000,
    src_vocab_size=3000,
    tgt_vocab_size=3000,
    share_vocab=True,
    lower=True
)
